In [2]:
#cleaned

# Imputation Method Selection based on Dataset Properties for Both Classification (Binary and Multiclass

In [3]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import seaborn as sns
from pandas.api.types import CategoricalDtype
from pathlib import Path

from numpy import mean
from numpy import std

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr

from sklearn.compose import make_column_transformer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

import tensorflow as tf

import csv
from sklearn.utils import Bunch
from sklearn import metrics
from math import sqrt

from numpy import loadtxt
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input
from keras.utils import to_categorical 

## Training

In [4]:
data_bin = pd.read_csv('binary_properties_train_dataset_original.csv')
data_multi = pd.read_csv('multi_properties_train_dataset_original.csv')

data = data_bin.append(data_multi, ignore_index=True)
data

C:\Users\Pascal\AppData\Local\Temp\ipykernel_15244\3071368399.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data_bin.append(data_multi, ignore_index=True)


,Imputation_Method,Missing Type,Missing Fraction,NumberOfInstances,NumberOfFeatures,NumberOfNumericFeatures,NumberOfCategoricalFeatures
0,Random Forest,MAR,0.01,39366.0,10.0,0.0,10.0
1,KNN,MAR,0.10,39366.0,10.0,0.0,10.0
2,Random Forest,MAR,0.30,39366.0,10.0,0.0,10.0
3,KNN,MAR,0.50,39366.0,10.0,0.0,10.0
4,VAE,MCAR,0.01,39366.0,10.0,0.0,10.0
...,...,...,...,...,...,...,...
571,GAIN,MCAR,0.50,20000.0,NaN,20.0,1.0
572,KNN,MNAR,0.01,20000.0,NaN,20.0,1.0
573,Discriminative DL,MNAR,0.10,20000.0,NaN,20.0,1.0
574,Mean/Mode,MNAR,0.30,20000.0,NaN,20.0,1.0


In [5]:
# OneHot-encoder -> multiple column approach
# 

transformer = make_column_transformer(
    (OneHotEncoder(), ['Imputation_Method', 'Missing Type']),
    remainder='passthrough')

transformed = transformer.fit_transform(data)
data_preprocessed = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
data_preprocessed


,onehotencoder__Imputation_Method_Discriminative DL,onehotencoder__Imputation_Method_GAIN,onehotencoder__Imputation_Method_KNN,onehotencoder__Imputation_Method_Mean/Mode,onehotencoder__Imputation_Method_Random Forest,onehotencoder__Imputation_Method_VAE,onehotencoder__Missing Type_MAR,onehotencoder__Missing Type_MCAR,onehotencoder__Missing Type_MNAR,remainder__Missing Fraction,remainder__NumberOfInstances,remainder__NumberOfFeatures,remainder__NumberOfNumericFeatures,remainder__NumberOfCategoricalFeatures
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.01,39366.0,10.0,0.0,10.0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.10,39366.0,10.0,0.0,10.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.30,39366.0,10.0,0.0,10.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.50,39366.0,10.0,0.0,10.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.01,39366.0,10.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.50,20000.0,NaN,20.0,1.0
572,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.01,20000.0,NaN,20.0,1.0
573,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.10,20000.0,NaN,20.0,1.0
574,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.30,20000.0,NaN,20.0,1.0


In [6]:
# Umwandlung in numpy arrays
X = data_preprocessed[['onehotencoder__Missing Type_MAR', 'onehotencoder__Missing Type_MCAR', 
                         'onehotencoder__Missing Type_MNAR', 'remainder__Missing Fraction', 
                         'remainder__NumberOfInstances',  
                         'remainder__NumberOfNumericFeatures', 'remainder__NumberOfCategoricalFeatures']].to_numpy()
#X = data_preprocessed_x
y = data[['Imputation_Method']].to_numpy()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
X_train.shape
#print(X_train)

(460, 7)

In [7]:
y_train.shape

(460, 1)

In [8]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train)
print(y)



[[-0.71982763  1.42348719 -0.6990516  ...  0.39086343 -1.41946984
   0.86423517]
 [ 1.38922147 -0.70250017 -0.6990516  ... -0.75834448 -0.31703547
  -0.46907342]
 [-0.71982763  1.42348719 -0.6990516  ... -0.23515901  0.7853989
  -0.46907342]
 ...
 [-0.71982763  1.42348719 -0.6990516  ...  0.61650152 -1.41946984
   1.53088947]
 [ 1.38922147 -0.70250017 -0.6990516  ...  0.39086343 -1.41946984
   0.86423517]
 [-0.71982763  1.42348719 -0.6990516  ...  0.03581736 -0.15954485
  -0.46907342]]
[['Random Forest']
 ['KNN']
 ['Random Forest']
 ['KNN']
 ['VAE']
 ['KNN']
 ['KNN']
 ['Random Forest']
 ['VAE']
 ['KNN']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['VAE']
 ['Discriminative DL']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['VAE']
 ['Mean/Mode']
 ['Mean/Mode']
 ['Random Forest']
 ['Discriminative DL']
 ['KNN']
 ['Mean/Mode']
 ['KNN']
 ['Random Forest']
 ['VAE']
 ['Mea

In [9]:
X_train.shape

(460, 7)

In [10]:
y.shape

(576, 1)

## Original Experiment Results

For the training we use the original experiment results here

### Random Forest with Nested Cross Validation, with GridSearch

In [17]:
X_train = X
y_train = y

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

outer_cv = KFold(n_splits=10, shuffle=True, random_state=42)
outer_results = list()
predicitons_list = list()
test_data_list = list()
ground_truth_list = list()

inner_nested_cv_results= []
inner_nested_best_score= []
inner_nested_best_params= []
inner_nested_test_score = []

for train_index, test_index in outer_cv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    clf = RandomForestClassifier(random_state=42, class_weight='balanced')

    param_grid = {
        'n_estimators': [50, 100, 250, 500, 1000, 3000, 6000, 10000, 20000],#, 3000, 4000, 5000, 6000, 7000],
#    'max_features': ['sqrt', 'log2'],
#    'criterion' :['gini', 'entropy'],
        'max_depth': [3,5,9,12,15,18,25,30]#,3,4,5,6,7]
    }

    grid_clf = GridSearchCV(clf, param_grid, cv=inner_cv)
    result = grid_clf.fit(X_train, y_train.ravel())
    best_model = result.best_estimator_
    print(grid_clf.best_params_)
    print(best_model)

    inner_nested_cv_results.append(grid_clf.cv_results_)
    inner_nested_best_score.append(grid_clf.best_score_)
    inner_nested_best_params.append(grid_clf.best_params_)
    inner_nested_test_score.append(grid_clf.score(X_test, y_test))

    yhat = best_model.predict(X_test)
#    print(yhat)
#    print('___________________----------\n')
#    print(X_test)
#    print('___________________----------\n')
#    print(y_test)
    acc = accuracy_score(y_test, yhat)
    outer_results.append(acc)
    predicitons_list.append(yhat)
    test_data_list.append(X_test)
    ground_truth_list.append(y_test)
    
   
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
    print('_____________________________')
    print('\n')
    
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))    


{'max_depth': 18, 'n_estimators': 50}
RandomForestClassifier(class_weight='balanced', max_depth=18, n_estimators=50,
                       random_state=42)
>acc=0.310, est=0.251, cfg={'max_depth': 18, 'n_estimators': 50}
_____________________________


{'max_depth': 5, 'n_estimators': 1000}
RandomForestClassifier(class_weight='balanced', max_depth=5, n_estimators=1000,
                       random_state=42)
>acc=0.259, est=0.251, cfg={'max_depth': 5, 'n_estimators': 1000}
_____________________________


{'max_depth': 9, 'n_estimators': 500}
RandomForestClassifier(class_weight='balanced', max_depth=9, n_estimators=500,
                       random_state=42)
>acc=0.276, est=0.257, cfg={'max_depth': 9, 'n_estimators': 500}
_____________________________


{'max_depth': 15, 'n_estimators': 50}
RandomForestClassifier(class_weight='balanced', max_depth=15, n_estimators=50,
                       random_state=42)
>acc=0.345, est=0.239, cfg={'max_depth': 15, 'n_estimators': 50}
_____________

In [18]:
print(type(yhat))
print(type(X_test))
print(type(y_test))
print(outer_results)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[0.3103448275862069, 0.25862068965517243, 0.27586206896551724, 0.3448275862068966, 0.22413793103448276, 0.20689655172413793, 0.3157894736842105, 0.21052631578947367, 0.21052631578947367, 0.3333333333333333]


In [19]:
#np.mean(inner_nested_test_score)

In [20]:
#inner_nested_cv_results

In [21]:
inner_nested_best_params

[{'max_depth': 18, 'n_estimators': 50},
 {'max_depth': 5, 'n_estimators': 1000},
 {'max_depth': 9, 'n_estimators': 500},
 {'max_depth': 15, 'n_estimators': 50},
 {'max_depth': 18, 'n_estimators': 100},
 {'max_depth': 15, 'n_estimators': 100},
 {'max_depth': 15, 'n_estimators': 100},
 {'max_depth': 9, 'n_estimators': 10000},
 {'max_depth': 12, 'n_estimators': 100},
 {'max_depth': 9, 'n_estimators': 3000}]

In [22]:
inner_nested_test_score

[0.3103448275862069,
 0.25862068965517243,
 0.27586206896551724,
 0.3448275862068966,
 0.22413793103448276,
 0.20689655172413793,
 0.3157894736842105,
 0.21052631578947367,
 0.21052631578947367,
 0.3333333333333333]

In [23]:
# Manually select the right model (best accuracy)

outer_results_item = outer_results[3]
print(outer_results_item)
df_predict_item = predicitons_list[3]
#df_predict_item
test_data_item = test_data_list[3]
ground_truth_item = ground_truth_list[3]

0.3448275862068966


In [24]:
df_predict = pd.DataFrame()
df_predict['prediction'] = df_predict_item
df_predict

,prediction
0,KNN
1,Random Forest
2,Random Forest
3,VAE
4,GAIN
5,Discriminative DL
6,VAE
7,Mean/Mode
8,VAE
9,Random Forest


In [25]:
df_testdata = pd.DataFrame({'MAR': test_data_item[:, 0], 'MCAR': test_data_item[:, 1], 'MNAR': test_data_item[:, 2], 'Missing Fraction': test_data_item[:, 3], 'NumberOfInstances': test_data_item[:, 4],'NumberOfNumericFeatures': test_data_item[:, 5], 'NumberOfCategoricalFeatures': test_data_item[:, 6]})
df_testdata


,MAR,MCAR,MNAR,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures
0,0.0,1.0,0.0,0.50,39366.0,0.0,10.0
1,0.0,1.0,0.0,0.10,45312.0,7.0,2.0
2,0.0,1.0,0.0,0.50,45312.0,7.0,2.0
3,1.0,0.0,0.0,0.01,39366.0,9.0,1.0
4,1.0,0.0,0.0,0.10,39366.0,9.0,1.0
5,0.0,1.0,0.0,0.50,39366.0,9.0,1.0
6,0.0,1.0,0.0,0.30,11183.0,6.0,1.0
7,1.0,0.0,0.0,0.10,8192.0,8.0,1.0
8,0.0,0.0,1.0,0.01,8192.0,8.0,1.0
9,0.0,0.0,1.0,0.10,8192.0,8.0,1.0


In [26]:
model_prediction = pd.DataFrame()
model_prediction = pd.concat([df_predict, df_testdata], axis=1)

#model_prediction.to_csv('model_prediction.csv', index=False)
model_prediction

,prediction,MAR,MCAR,MNAR,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures
0,KNN,0.0,1.0,0.0,0.50,39366.0,0.0,10.0
1,Random Forest,0.0,1.0,0.0,0.10,45312.0,7.0,2.0
2,Random Forest,0.0,1.0,0.0,0.50,45312.0,7.0,2.0
3,VAE,1.0,0.0,0.0,0.01,39366.0,9.0,1.0
4,GAIN,1.0,0.0,0.0,0.10,39366.0,9.0,1.0
5,Discriminative DL,0.0,1.0,0.0,0.50,39366.0,9.0,1.0
6,VAE,0.0,1.0,0.0,0.30,11183.0,6.0,1.0
7,Mean/Mode,1.0,0.0,0.0,0.10,8192.0,8.0,1.0
8,VAE,0.0,0.0,1.0,0.01,8192.0,8.0,1.0
9,Random Forest,0.0,0.0,1.0,0.10,8192.0,8.0,1.0


In [27]:
one_hot_reverse = model_prediction.copy()
one_hot_reverse = one_hot_reverse[['MAR', 'MCAR', 'MNAR']]

one_hot_reverse['Missing Type'] = one_hot_reverse.idxmax(1)
one_hot_reverse = one_hot_reverse[['Missing Type']]
one_hot_reverse

,Missing Type
0,MCAR
1,MCAR
2,MCAR
3,MAR
4,MAR
5,MCAR
6,MCAR
7,MAR
8,MNAR
9,MNAR


In [28]:
model_prediction = pd.concat([model_prediction, one_hot_reverse], axis=1)
model_prediction = model_prediction.drop(['MAR', 'MCAR', 'MNAR'], axis=1)
model_prediction['control'] = 20
model_prediction = model_prediction.rename(columns={"prediction": "Imputation_Method"})

model_prediction

,Imputation_Method,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,Missing Type,control
0,KNN,0.50,39366.0,0.0,10.0,MCAR,20
1,Random Forest,0.10,45312.0,7.0,2.0,MCAR,20
2,Random Forest,0.50,45312.0,7.0,2.0,MCAR,20
3,VAE,0.01,39366.0,9.0,1.0,MAR,20
4,GAIN,0.10,39366.0,9.0,1.0,MAR,20
5,Discriminative DL,0.50,39366.0,9.0,1.0,MCAR,20
6,VAE,0.30,11183.0,6.0,1.0,MCAR,20
7,Mean/Mode,0.10,8192.0,8.0,1.0,MAR,20
8,VAE,0.01,8192.0,8.0,1.0,MNAR,20
9,Random Forest,0.10,8192.0,8.0,1.0,MNAR,20


In [29]:
data_bin = pd.read_csv('binary_imputed_full_info.csv')
data_multi = pd.read_csv('multi_imputed_full_info.csv')

data_control = data_bin.append(data_multi, ignore_index=True)
data_control

C:\Users\Pascal\AppData\Local\Temp\ipykernel_15244\3705097935.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_control = data_bin.append(data_multi, ignore_index=True)


,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation,Data_Constellation_full,Performance Difference to Average Best
0,Random Forest,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674909,0.0,0.674760,...,13664.0,10.0,39366.0,0.0,10.0,NaN,1.0,MAR - 0.01,MAR - 0.01 - 137,0.000000e+00
1,KNN,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674729,0.0,0.674760,...,13664.0,10.0,39366.0,0.0,10.0,NaN,2.0,MAR - 0.01,MAR - 0.01 - 137,-2.949553e-08
2,Mean/Mode,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674145,0.0,0.674116,...,13664.0,10.0,39366.0,0.0,10.0,NaN,5.0,MAR - 0.01,MAR - 0.01 - 137,-6.439471e-04
3,VAE,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674729,0.0,0.674488,...,13664.0,10.0,39366.0,0.0,10.0,NaN,3.0,MAR - 0.01,MAR - 0.01 - 137,-2.716528e-04
4,Discriminative DL,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674505,0.0,0.674251,...,13664.0,10.0,39366.0,0.0,10.0,NaN,4.0,MAR - 0.01,MAR - 0.01 - 137,-5.093850e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3330,KNN,41671,MNAR,0.50,a9,downstream_performance_mean,F1_macro,0.264992,0.0,0.265395,...,743.0,21.0,20000.0,20.0,1.0,5.0,1.0,MNAR - 0.5,MNAR - 0.5 - 41671,2.477551e-02
3331,Mean/Mode,41671,MNAR,0.50,a9,downstream_performance_mean,F1_macro,0.239275,0.0,0.240780,...,743.0,21.0,20000.0,20.0,1.0,5.0,4.0,MNAR - 0.5,MNAR - 0.5 - 41671,1.601834e-04
3332,VAE,41671,MNAR,0.50,a9,downstream_performance_mean,F1_macro,0.241995,0.0,0.242421,...,743.0,21.0,20000.0,20.0,1.0,5.0,3.0,MNAR - 0.5,MNAR - 0.5 - 41671,1.801453e-03
3333,GAIN,41671,MNAR,0.50,a9,downstream_performance_mean,F1_macro,0.239877,0.0,0.243434,...,743.0,21.0,20000.0,20.0,1.0,5.0,2.0,MNAR - 0.5,MNAR - 0.5 - 41671,2.814915e-03


In [30]:
result_improv = pd.merge(model_prediction, data_control, how="left", on=["Imputation_Method", "Missing Fraction", "NumberOfInstances", "NumberOfNumericFeatures", "NumberOfCategoricalFeatures", "Missing Type"])
result_improv = result_improv[result_improv['Performance Difference to Average Best'].notna()]
#result_improv.to_csv('result_merge_control.csv')

result_improv

,Imputation_Method,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,Missing Type,control,Task,Column,result_type,...,Unnamed: 0,name,MajorityClassSize,MinorityClassSize,NumberOfFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation,Data_Constellation_full,Performance Difference to Average Best
0,KNN,0.50,39366.0,0.0,10.0,MCAR,20,137.0,top-middle-square,downstream_performance_mean,...,34.0,BNG(tic-tac-toe),25702.0,13664.0,10.0,NaN,4.0,MCAR - 0.5,MCAR - 0.5 - 137,-0.005791
1,Random Forest,0.10,45312.0,7.0,2.0,MCAR,20,151.0,nswprice,downstream_performance_mean,...,37.0,electricity,26075.0,19237.0,9.0,NaN,2.0,MCAR - 0.1,MCAR - 0.1 - 151,0.000000
2,Random Forest,0.50,45312.0,7.0,2.0,MCAR,20,151.0,nswprice,downstream_performance_mean,...,37.0,electricity,26075.0,19237.0,9.0,NaN,1.0,MCAR - 0.5,MCAR - 0.5 - 151,0.000000
3,VAE,0.01,39366.0,9.0,1.0,MAR,20,251.0,Cell_Shape_Uniformity,downstream_performance_mean,...,35.0,BNG(breast-w),25820.0,13546.0,10.0,NaN,2.0,MAR - 0.01,MAR - 0.01 - 251,0.000241
4,GAIN,0.10,39366.0,9.0,1.0,MAR,20,251.0,Cell_Shape_Uniformity,downstream_performance_mean,...,35.0,BNG(breast-w),25820.0,13546.0,10.0,NaN,6.0,MAR - 0.1,MAR - 0.1 - 251,-0.002556
5,Discriminative DL,0.50,39366.0,9.0,1.0,MCAR,20,251.0,Cell_Shape_Uniformity,downstream_performance_mean,...,35.0,BNG(breast-w),25820.0,13546.0,10.0,NaN,3.0,MCAR - 0.5,MCAR - 0.5 - 251,-0.000680
6,VAE,0.30,11183.0,6.0,1.0,MCAR,20,310.0,attr2,downstream_performance_mean,...,12.0,mammography,10923.0,260.0,7.0,NaN,1.0,MCAR - 0.3,MCAR - 0.3 - 310,0.023157
7,Mean/Mode,0.10,8192.0,8.0,1.0,MAR,20,725.0,a2pop,downstream_performance_mean,...,9.0,bank8FM,4885.0,3307.0,9.0,NaN,4.0,MAR - 0.1,MAR - 0.1 - 725,-0.001263
8,VAE,0.01,8192.0,8.0,1.0,MNAR,20,725.0,a2pop,downstream_performance_mean,...,9.0,bank8FM,4885.0,3307.0,9.0,NaN,1.0,MNAR - 0.01,MNAR - 0.01 - 725,0.007019
9,Random Forest,0.10,8192.0,8.0,1.0,MNAR,20,725.0,a2pop,downstream_performance_mean,...,9.0,bank8FM,4885.0,3307.0,9.0,NaN,1.0,MNAR - 0.1,MNAR - 0.1 - 725,0.000000


In [31]:
av_improv_model = result_improv['Performance Difference to Average Best'].mean()
print(av_improv_model)

0.004949344427307904


## Experiment Results adjusted for 0.01 F1 Score Points

For the training we use the experiment results, but replaced the best method for each data constellation with the average best method, if the average best method is not outscored by at least 0.01 F1 score points.

### Training

In [190]:

data_bin = pd.read_csv('binary_properties_train_dataset_1_percent.csv')
data_multi = pd.read_csv('multi_properties_train_dataset_1_percent.csv')

data = data_bin.append(data_multi, ignore_index=True)
data

C:\Users\Pascal\AppData\Local\Temp\ipykernel_12192\123082188.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data_bin.append(data_multi, ignore_index=True)


,Unnamed: 0,Imputation_Method,Missing Type,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures
0,999,Random Forest,MAR,0.01,15545.0,5.0,1.0
1,1071,Random Forest,MAR,0.01,19020.0,11.0,1.0
2,1140,Random Forest,MAR,0.01,39948.0,9.0,1.0
3,4,Random Forest,MAR,0.01,39366.0,0.0,10.0
4,1209,Random Forest,MAR,0.01,14980.0,14.0,1.0
...,...,...,...,...,...,...,...
559,1070,VAE,MNAR,0.50,58000.0,9.0,1.0
560,1140,Random Forest,MNAR,0.50,44819.0,6.0,1.0
561,1209,KNN,MNAR,0.50,20000.0,20.0,1.0
562,786,Mean/Mode,MNAR,0.50,5665.0,2.0,15.0


In [191]:
# OneHot-encoder -> multiple column approach
# 

transformer = make_column_transformer(
    (OneHotEncoder(), ['Imputation_Method', 'Missing Type']),
    remainder='passthrough')

transformed = transformer.fit_transform(data)
data_preprocessed = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
data_preprocessed


,onehotencoder__Imputation_Method_Discriminative DL,onehotencoder__Imputation_Method_GAIN,onehotencoder__Imputation_Method_KNN,onehotencoder__Imputation_Method_Mean/Mode,onehotencoder__Imputation_Method_Random Forest,onehotencoder__Imputation_Method_VAE,onehotencoder__Missing Type_MAR,onehotencoder__Missing Type_MCAR,onehotencoder__Missing Type_MNAR,remainder__Unnamed: 0,remainder__Missing Fraction,remainder__NumberOfInstances,remainder__NumberOfNumericFeatures,remainder__NumberOfCategoricalFeatures
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,999.0,0.01,15545.0,5.0,1.0
1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1071.0,0.01,19020.0,11.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1140.0,0.01,39948.0,9.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0,0.01,39366.0,0.0,10.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1209.0,0.01,14980.0,14.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1070.0,0.50,58000.0,9.0,1.0
560,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1140.0,0.50,44819.0,6.0,1.0
561,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1209.0,0.50,20000.0,20.0,1.0
562,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,786.0,0.50,5665.0,2.0,15.0


In [192]:
# Umwandlung in numpy arrays
X = data_preprocessed[['onehotencoder__Missing Type_MAR', 'onehotencoder__Missing Type_MCAR', 
                         'onehotencoder__Missing Type_MNAR', 'remainder__Missing Fraction', 
                         'remainder__NumberOfInstances',  
                         'remainder__NumberOfNumericFeatures', 'remainder__NumberOfCategoricalFeatures']].to_numpy()
#X = data_preprocessed_x
y = data[['Imputation_Method']].to_numpy()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
X_train.shape
#print(X_train)

(451, 7)

In [193]:
y_train.shape

(451, 1)

In [194]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train)
print(y)



[[-0.68487617 -0.73433585  1.42367329 ... -0.01648855  0.267639
  -0.45201032]
 [ 1.46011796 -0.73433585 -0.70240835 ...  1.04613833 -0.36349924
   0.20559179]
 [-0.68487617  1.36177472 -0.70240835 ... -0.01648855  0.267639
  -0.45201032]
 ...
 [-0.68487617 -0.73433585  1.42367329 ... -0.18628018 -0.67906836
  -0.45201032]
 [-0.68487617  1.36177472 -0.70240835 ... -0.40874387  1.0565618
  -0.45201032]
 [ 1.46011796 -0.73433585 -0.70240835 ... -0.39941143 -0.5212838
  -0.45201032]]
[['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['VAE']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random F

In [195]:
X_train.shape

(451, 7)

In [196]:
y.shape

(564, 1)

### Random Forest with Nested Cross Validation, with GridSearch

In [203]:
X_train = X
y_train = y

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

outer_cv = KFold(n_splits=10, shuffle=True, random_state=42)
outer_results_1 = list()
predicitons_list_1 = list()
test_data_list_1 = list()
ground_truth_list_1 = list()

inner_nested_cv_results= []
inner_nested_best_score= []
inner_nested_best_params= []
inner_nested_test_score = []

for train_index, test_index in outer_cv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    clf = RandomForestClassifier(random_state=42, class_weight='balanced')

    param_grid = {
        'n_estimators': [100, 500, 1000, 5000, 10000, 30000],#, 3000, 4000, 5000, 6000, 7000],
#    'max_features': ['sqrt', 'log2'],
#    'criterion' :['gini', 'entropy'],
        'max_depth': [1,9,15,21,50]#,3,4,5,6,7]
    }

    grid_clf = GridSearchCV(clf, param_grid, cv=inner_cv)
    result = grid_clf.fit(X_train, y_train.ravel())
    best_model = result.best_estimator_
    print(grid_clf.best_params_)
    print(best_model)

    inner_nested_cv_results.append(grid_clf.cv_results_)
    inner_nested_best_score.append(grid_clf.best_score_)
    inner_nested_best_params.append(grid_clf.best_params_)
    inner_nested_test_score.append(grid_clf.score(X_test, y_test))

    yhat = best_model.predict(X_test)
#    print(yhat)
#    print('___________________----------\n')
#    print(X_test)
#    print('___________________----------\n')
#    print(y_test)
    acc = accuracy_score(y_test, yhat)
    outer_results_1.append(acc)
    predicitons_list_1.append(yhat)
    test_data_list_1.append(X_test)
    ground_truth_list_1.append(y_test)
    
   
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
    print('_____________________________')
    print('\n')
    
print('Accuracy: %.3f (%.3f)' % (mean(outer_results_1), std(outer_results_1)))    


{'max_depth': 15, 'n_estimators': 30000}
RandomForestClassifier(class_weight='balanced', max_depth=15,
                       n_estimators=30000, random_state=42)
>acc=0.684, est=0.718, cfg={'max_depth': 15, 'n_estimators': 30000}
_____________________________


{'max_depth': 21, 'n_estimators': 100}
RandomForestClassifier(class_weight='balanced', max_depth=21, random_state=42)
>acc=0.719, est=0.706, cfg={'max_depth': 21, 'n_estimators': 100}
_____________________________


{'max_depth': 15, 'n_estimators': 30000}
RandomForestClassifier(class_weight='balanced', max_depth=15,
                       n_estimators=30000, random_state=42)
>acc=0.702, est=0.698, cfg={'max_depth': 15, 'n_estimators': 30000}
_____________________________


{'max_depth': 9, 'n_estimators': 30000}
RandomForestClassifier(class_weight='balanced', max_depth=9, n_estimators=30000,
                       random_state=42)
>acc=0.719, est=0.700, cfg={'max_depth': 9, 'n_estimators': 30000}
_____________________________


In [204]:
print(type(yhat))
print(type(X_test))
print(type(y_test))
print(outer_results_1)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[0.6842105263157895, 0.7192982456140351, 0.7017543859649122, 0.7192982456140351, 0.6964285714285714, 0.7142857142857143, 0.6428571428571429, 0.75, 0.7321428571428571, 0.6607142857142857]


In [205]:
#np.mean(inner_nested_test_score)

In [206]:
#inner_nested_cv_results

In [207]:
inner_nested_best_params

[{'max_depth': 15, 'n_estimators': 30000},
 {'max_depth': 21, 'n_estimators': 100},
 {'max_depth': 15, 'n_estimators': 30000},
 {'max_depth': 9, 'n_estimators': 30000},
 {'max_depth': 15, 'n_estimators': 5000},
 {'max_depth': 15, 'n_estimators': 500},
 {'max_depth': 15, 'n_estimators': 10000},
 {'max_depth': 15, 'n_estimators': 30000},
 {'max_depth': 15, 'n_estimators': 500},
 {'max_depth': 15, 'n_estimators': 5000}]

In [208]:
inner_nested_test_score

[0.6842105263157895,
 0.7192982456140351,
 0.7017543859649122,
 0.7192982456140351,
 0.6964285714285714,
 0.7142857142857143,
 0.6428571428571429,
 0.75,
 0.7321428571428571,
 0.6607142857142857]

In [209]:
# Manually select the right model (best accuracy)

outer_results_item = outer_results_1[7]
print(outer_results_item)
df_predict_item = predicitons_list_1[7]
#df_predict_item
test_data_item = test_data_list_1[7]
ground_truth_item = ground_truth_list_1[7]

0.75


In [210]:
df_predict = pd.DataFrame()
df_predict['prediction'] = df_predict_item
df_predict

,prediction
0,Random Forest
1,Random Forest
2,Random Forest
3,Random Forest
4,Random Forest
5,Random Forest
6,Random Forest
7,Random Forest
8,Random Forest
9,Random Forest


In [211]:
df_testdata = pd.DataFrame({'MAR': test_data_item[:, 0], 'MCAR': test_data_item[:, 1], 'MNAR': test_data_item[:, 2], 'Missing Fraction': test_data_item[:, 3], 'NumberOfInstances': test_data_item[:, 4],'NumberOfNumericFeatures': test_data_item[:, 5], 'NumberOfCategoricalFeatures': test_data_item[:, 6]})
df_testdata


,MAR,MCAR,MNAR,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures
0,1.0,0.0,0.0,0.01,14980.0,14.0,1.0
1,1.0,0.0,0.0,0.01,11183.0,6.0,1.0
2,1.0,0.0,0.0,0.01,40768.0,7.0,4.0
3,1.0,0.0,0.0,0.01,40768.0,10.0,1.0
4,1.0,0.0,0.0,0.10,39948.0,9.0,1.0
5,1.0,0.0,0.0,0.10,5404.0,5.0,1.0
6,1.0,0.0,0.0,0.10,39366.0,9.0,1.0
7,1.0,0.0,0.0,0.10,5278.0,7.0,7.0
8,1.0,0.0,0.0,0.10,4052.0,7.0,1.0
9,1.0,0.0,0.0,0.30,19020.0,11.0,1.0


In [212]:
model_prediction = pd.DataFrame()
model_prediction = pd.concat([df_predict, df_testdata], axis=1)

#model_prediction.to_csv('model_prediction.csv', index=False)
model_prediction

,prediction,MAR,MCAR,MNAR,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures
0,Random Forest,1.0,0.0,0.0,0.01,14980.0,14.0,1.0
1,Random Forest,1.0,0.0,0.0,0.01,11183.0,6.0,1.0
2,Random Forest,1.0,0.0,0.0,0.01,40768.0,7.0,4.0
3,Random Forest,1.0,0.0,0.0,0.01,40768.0,10.0,1.0
4,Random Forest,1.0,0.0,0.0,0.10,39948.0,9.0,1.0
5,Random Forest,1.0,0.0,0.0,0.10,5404.0,5.0,1.0
6,Random Forest,1.0,0.0,0.0,0.10,39366.0,9.0,1.0
7,Random Forest,1.0,0.0,0.0,0.10,5278.0,7.0,7.0
8,Random Forest,1.0,0.0,0.0,0.10,4052.0,7.0,1.0
9,Random Forest,1.0,0.0,0.0,0.30,19020.0,11.0,1.0


In [213]:
one_hot_reverse = model_prediction.copy()
one_hot_reverse = one_hot_reverse[['MAR', 'MCAR', 'MNAR']]

one_hot_reverse['Missing Type'] = one_hot_reverse.idxmax(1)
one_hot_reverse = one_hot_reverse[['Missing Type']]
one_hot_reverse

,Missing Type
0,MAR
1,MAR
2,MAR
3,MAR
4,MAR
5,MAR
6,MAR
7,MAR
8,MAR
9,MAR


In [214]:
model_prediction = pd.concat([model_prediction, one_hot_reverse], axis=1)
model_prediction = model_prediction.drop(['MAR', 'MCAR', 'MNAR'], axis=1)
model_prediction['control'] = 20
model_prediction = model_prediction.rename(columns={"prediction": "Imputation_Method"})

model_prediction

,Imputation_Method,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,Missing Type,control
0,Random Forest,0.01,14980.0,14.0,1.0,MAR,20
1,Random Forest,0.01,11183.0,6.0,1.0,MAR,20
2,Random Forest,0.01,40768.0,7.0,4.0,MAR,20
3,Random Forest,0.01,40768.0,10.0,1.0,MAR,20
4,Random Forest,0.10,39948.0,9.0,1.0,MAR,20
5,Random Forest,0.10,5404.0,5.0,1.0,MAR,20
6,Random Forest,0.10,39366.0,9.0,1.0,MAR,20
7,Random Forest,0.10,5278.0,7.0,7.0,MAR,20
8,Random Forest,0.10,4052.0,7.0,1.0,MAR,20
9,Random Forest,0.30,19020.0,11.0,1.0,MAR,20


In [215]:
data_bin = pd.read_csv('binary_imputed_full_info.csv')
data_multi = pd.read_csv('multi_imputed_full_info.csv')

data_control = data_bin.append(data_multi, ignore_index=True)
data_control

C:\Users\Pascal\AppData\Local\Temp\ipykernel_12192\3705097935.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_control = data_bin.append(data_multi, ignore_index=True)


,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation,Data_Constellation_full,Performance Difference to Average Best
0,Random Forest,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674909,0.0,0.674760,...,13664.0,10.0,39366.0,0.0,10.0,NaN,1.0,MAR - 0.01,MAR - 0.01 - 137,0.000000e+00
1,KNN,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674729,0.0,0.674760,...,13664.0,10.0,39366.0,0.0,10.0,NaN,2.0,MAR - 0.01,MAR - 0.01 - 137,-2.949553e-08
2,Mean/Mode,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674145,0.0,0.674116,...,13664.0,10.0,39366.0,0.0,10.0,NaN,5.0,MAR - 0.01,MAR - 0.01 - 137,-6.439471e-04
3,VAE,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674729,0.0,0.674488,...,13664.0,10.0,39366.0,0.0,10.0,NaN,3.0,MAR - 0.01,MAR - 0.01 - 137,-2.716528e-04
4,Discriminative DL,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674505,0.0,0.674251,...,13664.0,10.0,39366.0,0.0,10.0,NaN,4.0,MAR - 0.01,MAR - 0.01 - 137,-5.093850e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3330,KNN,41671,MNAR,0.50,a9,downstream_performance_mean,F1_macro,0.264992,0.0,0.265395,...,743.0,21.0,20000.0,20.0,1.0,5.0,1.0,MNAR - 0.5,MNAR - 0.5 - 41671,2.477551e-02
3331,Mean/Mode,41671,MNAR,0.50,a9,downstream_performance_mean,F1_macro,0.239275,0.0,0.240780,...,743.0,21.0,20000.0,20.0,1.0,5.0,4.0,MNAR - 0.5,MNAR - 0.5 - 41671,1.601834e-04
3332,VAE,41671,MNAR,0.50,a9,downstream_performance_mean,F1_macro,0.241995,0.0,0.242421,...,743.0,21.0,20000.0,20.0,1.0,5.0,3.0,MNAR - 0.5,MNAR - 0.5 - 41671,1.801453e-03
3333,GAIN,41671,MNAR,0.50,a9,downstream_performance_mean,F1_macro,0.239877,0.0,0.243434,...,743.0,21.0,20000.0,20.0,1.0,5.0,2.0,MNAR - 0.5,MNAR - 0.5 - 41671,2.814915e-03


In [216]:
result_improv = pd.merge(model_prediction, data_control, how="left", on=["Imputation_Method", "Missing Fraction", "NumberOfInstances", "NumberOfNumericFeatures", "NumberOfCategoricalFeatures", "Missing Type"])
result_improv = result_improv[result_improv['Performance Difference to Average Best'].notna()]
#result_improv.to_csv('result_merge_control.csv')

result_improv

,Imputation_Method,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,Missing Type,control,Task,Column,result_type,...,Unnamed: 0,name,MajorityClassSize,MinorityClassSize,NumberOfFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation,Data_Constellation_full,Performance Difference to Average Best
0,Random Forest,0.01,14980.0,14.0,1.0,MAR,20,1471,V9,downstream_performance_mean,...,28,eeg-eye-state,8257.0,6723.0,15.0,NaN,3.0,MAR - 0.01,MAR - 0.01 - 1471,0.000000
1,Random Forest,0.01,11183.0,6.0,1.0,MAR,20,310,attr2,downstream_performance_mean,...,12,mammography,10923.0,260.0,7.0,NaN,2.0,MAR - 0.01,MAR - 0.01 - 310,0.000000
2,Random Forest,0.01,40768.0,7.0,4.0,MAR,20,881,x3,downstream_performance_mean,...,39,mv,24321.0,16447.0,11.0,NaN,3.0,MAR - 0.01,MAR - 0.01 - 881,0.000000
3,Random Forest,0.01,40768.0,10.0,1.0,MAR,20,901,X2,downstream_performance_mean,...,38,fried,20427.0,20341.0,11.0,NaN,3.0,MAR - 0.01,MAR - 0.01 - 901,0.000000
4,Random Forest,0.10,39948.0,9.0,1.0,MAR,20,1220,keyword_id,downstream_performance_mean,...,36,Click_prediction_small,33220.0,6728.0,10.0,NaN,2.0,MAR - 0.1,MAR - 0.1 - 1220,0.000000
5,Random Forest,0.10,5404.0,5.0,1.0,MAR,20,1489,V1,downstream_performance_mean,...,4,phoneme,3818.0,1586.0,6.0,NaN,2.0,MAR - 0.1,MAR - 0.1 - 1489,0.000000
6,Random Forest,0.10,39366.0,9.0,1.0,MAR,20,251,Cell_Shape_Uniformity,downstream_performance_mean,...,35,BNG(breast-w),25820.0,13546.0,10.0,NaN,4.0,MAR - 0.1,MAR - 0.1 - 251,0.000000
7,Random Forest,0.10,5278.0,7.0,7.0,MAR,20,42192,age,downstream_performance_mean,...,10,compas-two-years,2795.0,2483.0,14.0,NaN,2.0,MAR - 0.1,MAR - 0.1 - 42192,0.000000
8,Random Forest,0.10,4052.0,7.0,1.0,MAR,20,728,Lower_court_disagreement,downstream_performance_mean,...,3,analcatdata_supreme,3081.0,971.0,8.0,NaN,1.0,MAR - 0.1,MAR - 0.1 - 728,0.000000
9,Random Forest,0.30,19020.0,11.0,1.0,MAR,20,1120,fAlpha_,downstream_performance_mean,...,29,MagicTelescope,12332.0,6688.0,12.0,NaN,1.0,MAR - 0.3,MAR - 0.3 - 1120,0.000000


In [217]:
av_improv_model = result_improv['Performance Difference to Average Best'].mean()
print(av_improv_model)

0.0009571587515765703


## Experiment Results adjusted for 0.03 F1 Score Points

For the training we use the experiment results, but replaced the best method for each data constellation with the average best method, if the average best method is not outscored by at least 0.03 F1 score points.

### Training

In [122]:
data_bin = pd.read_csv('binary_properties_train_dataset_3_percent.csv')
data_multi = pd.read_csv('multi_properties_train_dataset_3_percent.csv')

data = data_bin.append(data_multi, ignore_index=True)
data


C:\Users\Pascal\AppData\Local\Temp\ipykernel_12192\1271601156.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data_bin.append(data_multi, ignore_index=True)


,Unnamed: 0,Imputation_Method,Missing Type,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures
0,999,Random Forest,MAR,0.01,15545.0,5.0,1.0
1,1071,Random Forest,MAR,0.01,19020.0,11.0,1.0
2,1140,Random Forest,MAR,0.01,39948.0,9.0,1.0
3,4,Random Forest,MAR,0.01,39366.0,0.0,10.0
4,1209,Random Forest,MAR,0.01,14980.0,14.0,1.0
...,...,...,...,...,...,...,...
559,1067,Random Forest,MNAR,0.50,58000.0,9.0,1.0
560,1140,Random Forest,MNAR,0.50,44819.0,6.0,1.0
561,1204,Random Forest,MNAR,0.50,20000.0,20.0,1.0
562,784,Random Forest,MNAR,0.50,5665.0,2.0,15.0


In [123]:
# OneHot-encoder -> multiple column approach
# 

transformer = make_column_transformer(
    (OneHotEncoder(), ['Imputation_Method', 'Missing Type']),
    remainder='passthrough')

transformed = transformer.fit_transform(data)
data_preprocessed = pd.DataFrame(transformed, columns=transformer.get_feature_names_out())
data_preprocessed


,onehotencoder__Imputation_Method_Discriminative DL,onehotencoder__Imputation_Method_GAIN,onehotencoder__Imputation_Method_KNN,onehotencoder__Imputation_Method_Mean/Mode,onehotencoder__Imputation_Method_Random Forest,onehotencoder__Imputation_Method_VAE,onehotencoder__Missing Type_MAR,onehotencoder__Missing Type_MCAR,onehotencoder__Missing Type_MNAR,remainder__Unnamed: 0,remainder__Missing Fraction,remainder__NumberOfInstances,remainder__NumberOfNumericFeatures,remainder__NumberOfCategoricalFeatures
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,999.0,0.01,15545.0,5.0,1.0
1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1071.0,0.01,19020.0,11.0,1.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1140.0,0.01,39948.0,9.0,1.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,4.0,0.01,39366.0,0.0,10.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1209.0,0.01,14980.0,14.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1067.0,0.50,58000.0,9.0,1.0
560,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1140.0,0.50,44819.0,6.0,1.0
561,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1204.0,0.50,20000.0,20.0,1.0
562,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,784.0,0.50,5665.0,2.0,15.0


In [124]:
# Umwandlung in numpy arrays
X = data_preprocessed[['onehotencoder__Missing Type_MAR', 'onehotencoder__Missing Type_MCAR', 
                         'onehotencoder__Missing Type_MNAR', 'remainder__Missing Fraction', 
                         'remainder__NumberOfInstances',  
                         'remainder__NumberOfNumericFeatures', 'remainder__NumberOfCategoricalFeatures']].to_numpy()
#X = data_preprocessed_x
y = data[['Imputation_Method']].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
X_train.shape
#print(X_train)

(451, 7)

In [125]:
y_train.shape

(451, 1)

In [126]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train)
print(y)



[[-0.68487617 -0.73433585  1.42367329 ... -0.01648855  0.267639
  -0.45201032]
 [ 1.46011796 -0.73433585 -0.70240835 ...  1.04613833 -0.36349924
   0.20559179]
 [-0.68487617  1.36177472 -0.70240835 ... -0.01648855  0.267639
  -0.45201032]
 ...
 [-0.68487617 -0.73433585  1.42367329 ... -0.18628018 -0.67906836
  -0.45201032]
 [-0.68487617  1.36177472 -0.70240835 ... -0.40874387  1.0565618
  -0.45201032]
 [ 1.46011796 -0.73433585 -0.70240835 ... -0.39941143 -0.5212838
  -0.45201032]]
[['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['VAE']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random Forest']
 ['Random F

In [127]:
X_train.shape

(451, 7)

In [128]:
y.shape

(564, 1)

### Random Forest with Nested Cross Validation, with GridSearch

In [135]:
X_train = X
y_train = y

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

outer_cv = KFold(n_splits=10, shuffle=True, random_state=42)
outer_results = list()
predicitons_list = list()
test_data_list = list()
ground_truth_list = list()

inner_nested_cv_results= []
inner_nested_best_score= []
inner_nested_best_params= []
inner_nested_test_score = []

for train_index, test_index in outer_cv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    inner_cv = KFold(n_splits=5, shuffle=True, random_state=42)
    
    clf = RandomForestClassifier(random_state=42, class_weight='balanced')

    param_grid = {
        'n_estimators': [25, 50, 100, 250, 500, 1000, 3000, 4500, 6000, 10000],#, 3000, 4000, 5000, 6000, 7000],
#    'max_features': ['sqrt', 'log2'],
#    'criterion' :['gini', 'entropy'],
        'max_depth': [6,9,12,15,18]#,3,4,5,6,7]
    }

    grid_clf = GridSearchCV(clf, param_grid, cv=inner_cv)
    result = grid_clf.fit(X_train, y_train.ravel())
    best_model = result.best_estimator_
    print(grid_clf.best_params_)
    print(best_model)

    inner_nested_cv_results.append(grid_clf.cv_results_)
    inner_nested_best_score.append(grid_clf.best_score_)
    inner_nested_best_params.append(grid_clf.best_params_)
    inner_nested_test_score.append(grid_clf.score(X_test, y_test))

    yhat = best_model.predict(X_test)
#    print(yhat)
#    print('___________________----------\n')
#    print(X_test)
#    print('___________________----------\n')
#    print(y_test)
    acc = accuracy_score(y_test, yhat)
    outer_results.append(acc)
    predicitons_list.append(yhat)
    test_data_list.append(X_test)
    ground_truth_list.append(y_test)
    
   
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
    print('_____________________________')
    print('\n')
    
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))    


{'max_depth': 12, 'n_estimators': 100}
RandomForestClassifier(class_weight='balanced', max_depth=12, random_state=42)
>acc=0.860, est=0.890, cfg={'max_depth': 12, 'n_estimators': 100}
_____________________________


{'max_depth': 12, 'n_estimators': 250}
RandomForestClassifier(class_weight='balanced', max_depth=12, n_estimators=250,
                       random_state=42)
>acc=0.930, est=0.880, cfg={'max_depth': 12, 'n_estimators': 250}
_____________________________


{'max_depth': 12, 'n_estimators': 250}
RandomForestClassifier(class_weight='balanced', max_depth=12, n_estimators=250,
                       random_state=42)
>acc=0.807, est=0.888, cfg={'max_depth': 12, 'n_estimators': 250}
_____________________________


{'max_depth': 9, 'n_estimators': 25}
RandomForestClassifier(class_weight='balanced', max_depth=9, n_estimators=25,
                       random_state=42)
>acc=0.930, est=0.880, cfg={'max_depth': 9, 'n_estimators': 25}
_____________________________


{'max_depth': 12, '

In [136]:
print(type(yhat))
print(type(X_test))
print(type(y_test))
print(outer_results)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[0.8596491228070176, 0.9298245614035088, 0.8070175438596491, 0.9298245614035088, 0.8928571428571429, 0.9285714285714286, 0.8214285714285714, 0.9464285714285714, 0.8928571428571429, 0.8214285714285714]


In [137]:
#np.mean(inner_nested_test_score)

In [138]:
#inner_nested_cv_results

In [139]:
inner_nested_best_params

[{'max_depth': 12, 'n_estimators': 100},
 {'max_depth': 12, 'n_estimators': 250},
 {'max_depth': 12, 'n_estimators': 250},
 {'max_depth': 9, 'n_estimators': 25},
 {'max_depth': 12, 'n_estimators': 25},
 {'max_depth': 9, 'n_estimators': 500},
 {'max_depth': 12, 'n_estimators': 3000},
 {'max_depth': 12, 'n_estimators': 500},
 {'max_depth': 15, 'n_estimators': 100},
 {'max_depth': 12, 'n_estimators': 6000}]

In [140]:
inner_nested_test_score

[0.8596491228070176,
 0.9298245614035088,
 0.8070175438596491,
 0.9298245614035088,
 0.8928571428571429,
 0.9285714285714286,
 0.8214285714285714,
 0.9464285714285714,
 0.8928571428571429,
 0.8214285714285714]

In [151]:
# Manually select the right model (best accuracy)

outer_results_item = outer_results[7]
print(outer_results_item)
df_predict_item = predicitons_list[7]
#df_predict_item
test_data_item = test_data_list[7]
ground_truth_item = ground_truth_list[7]

0.9464285714285714


In [152]:
df_predict = pd.DataFrame()
df_predict['prediction'] = df_predict_item
df_predict

,prediction
0,Random Forest
1,Random Forest
2,Random Forest
3,Random Forest
4,Random Forest
5,Random Forest
6,Random Forest
7,Random Forest
8,Random Forest
9,Random Forest


In [153]:
df_testdata = pd.DataFrame({'MAR': test_data_item[:, 0], 'MCAR': test_data_item[:, 1], 'MNAR': test_data_item[:, 2], 'Missing Fraction': test_data_item[:, 3], 'NumberOfInstances': test_data_item[:, 4],'NumberOfNumericFeatures': test_data_item[:, 5], 'NumberOfCategoricalFeatures': test_data_item[:, 6]})
df_testdata


,MAR,MCAR,MNAR,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures
0,1.0,0.0,0.0,0.01,14980.0,14.0,1.0
1,1.0,0.0,0.0,0.01,11183.0,6.0,1.0
2,1.0,0.0,0.0,0.01,40768.0,7.0,4.0
3,1.0,0.0,0.0,0.01,40768.0,10.0,1.0
4,1.0,0.0,0.0,0.10,39948.0,9.0,1.0
5,1.0,0.0,0.0,0.10,5404.0,5.0,1.0
6,1.0,0.0,0.0,0.10,39366.0,9.0,1.0
7,1.0,0.0,0.0,0.10,5278.0,7.0,7.0
8,1.0,0.0,0.0,0.10,4052.0,7.0,1.0
9,1.0,0.0,0.0,0.30,19020.0,11.0,1.0


In [154]:
model_prediction = pd.DataFrame()
model_prediction = pd.concat([df_predict, df_testdata], axis=1)

#model_prediction.to_csv('model_prediction.csv', index=False)
model_prediction

,prediction,MAR,MCAR,MNAR,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures
0,Random Forest,1.0,0.0,0.0,0.01,14980.0,14.0,1.0
1,Random Forest,1.0,0.0,0.0,0.01,11183.0,6.0,1.0
2,Random Forest,1.0,0.0,0.0,0.01,40768.0,7.0,4.0
3,Random Forest,1.0,0.0,0.0,0.01,40768.0,10.0,1.0
4,Random Forest,1.0,0.0,0.0,0.10,39948.0,9.0,1.0
5,Random Forest,1.0,0.0,0.0,0.10,5404.0,5.0,1.0
6,Random Forest,1.0,0.0,0.0,0.10,39366.0,9.0,1.0
7,Random Forest,1.0,0.0,0.0,0.10,5278.0,7.0,7.0
8,Random Forest,1.0,0.0,0.0,0.10,4052.0,7.0,1.0
9,Random Forest,1.0,0.0,0.0,0.30,19020.0,11.0,1.0


In [155]:
one_hot_reverse = model_prediction.copy()
one_hot_reverse = one_hot_reverse[['MAR', 'MCAR', 'MNAR']]

one_hot_reverse['Missing Type'] = one_hot_reverse.idxmax(1)
one_hot_reverse = one_hot_reverse[['Missing Type']]
one_hot_reverse

,Missing Type
0,MAR
1,MAR
2,MAR
3,MAR
4,MAR
5,MAR
6,MAR
7,MAR
8,MAR
9,MAR


In [156]:
model_prediction = pd.concat([model_prediction, one_hot_reverse], axis=1)
model_prediction = model_prediction.drop(['MAR', 'MCAR', 'MNAR'], axis=1)
model_prediction['control'] = 20
model_prediction = model_prediction.rename(columns={"prediction": "Imputation_Method"})

model_prediction

,Imputation_Method,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,Missing Type,control
0,Random Forest,0.01,14980.0,14.0,1.0,MAR,20
1,Random Forest,0.01,11183.0,6.0,1.0,MAR,20
2,Random Forest,0.01,40768.0,7.0,4.0,MAR,20
3,Random Forest,0.01,40768.0,10.0,1.0,MAR,20
4,Random Forest,0.10,39948.0,9.0,1.0,MAR,20
5,Random Forest,0.10,5404.0,5.0,1.0,MAR,20
6,Random Forest,0.10,39366.0,9.0,1.0,MAR,20
7,Random Forest,0.10,5278.0,7.0,7.0,MAR,20
8,Random Forest,0.10,4052.0,7.0,1.0,MAR,20
9,Random Forest,0.30,19020.0,11.0,1.0,MAR,20


In [157]:
data_bin = pd.read_csv('binary_imputed_full_info.csv')
data_multi = pd.read_csv('multi_imputed_full_info.csv')

data_control = data_bin.append(data_multi, ignore_index=True)
data_control

C:\Users\Pascal\AppData\Local\Temp\ipykernel_12192\3705097935.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_control = data_bin.append(data_multi, ignore_index=True)


,Imputation_Method,Task,Missing Type,Missing Fraction,Column,result_type,metric,Baseline,Corrupted,Imputed,...,MinorityClassSize,NumberOfFeatures,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation,Data_Constellation_full,Performance Difference to Average Best
0,Random Forest,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674909,0.0,0.674760,...,13664.0,10.0,39366.0,0.0,10.0,NaN,1.0,MAR - 0.01,MAR - 0.01 - 137,0.000000e+00
1,KNN,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674729,0.0,0.674760,...,13664.0,10.0,39366.0,0.0,10.0,NaN,2.0,MAR - 0.01,MAR - 0.01 - 137,-2.949553e-08
2,Mean/Mode,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674145,0.0,0.674116,...,13664.0,10.0,39366.0,0.0,10.0,NaN,5.0,MAR - 0.01,MAR - 0.01 - 137,-6.439471e-04
3,VAE,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674729,0.0,0.674488,...,13664.0,10.0,39366.0,0.0,10.0,NaN,3.0,MAR - 0.01,MAR - 0.01 - 137,-2.716528e-04
4,Discriminative DL,137,MAR,0.01,top-middle-square,downstream_performance_mean,Classification Tasks,0.674505,0.0,0.674251,...,13664.0,10.0,39366.0,0.0,10.0,NaN,4.0,MAR - 0.01,MAR - 0.01 - 137,-5.093850e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3330,KNN,41671,MNAR,0.50,a9,downstream_performance_mean,F1_macro,0.264992,0.0,0.265395,...,743.0,21.0,20000.0,20.0,1.0,5.0,1.0,MNAR - 0.5,MNAR - 0.5 - 41671,2.477551e-02
3331,Mean/Mode,41671,MNAR,0.50,a9,downstream_performance_mean,F1_macro,0.239275,0.0,0.240780,...,743.0,21.0,20000.0,20.0,1.0,5.0,4.0,MNAR - 0.5,MNAR - 0.5 - 41671,1.601834e-04
3332,VAE,41671,MNAR,0.50,a9,downstream_performance_mean,F1_macro,0.241995,0.0,0.242421,...,743.0,21.0,20000.0,20.0,1.0,5.0,3.0,MNAR - 0.5,MNAR - 0.5 - 41671,1.801453e-03
3333,GAIN,41671,MNAR,0.50,a9,downstream_performance_mean,F1_macro,0.239877,0.0,0.243434,...,743.0,21.0,20000.0,20.0,1.0,5.0,2.0,MNAR - 0.5,MNAR - 0.5 - 41671,2.814915e-03


In [158]:
result_improv = pd.merge(model_prediction, data_control, how="left", on=["Imputation_Method", "Missing Fraction", "NumberOfInstances", "NumberOfNumericFeatures", "NumberOfCategoricalFeatures", "Missing Type"])
result_improv = result_improv[result_improv['Performance Difference to Average Best'].notna()]
#result_improv.to_csv('result_merge_control.csv')

result_improv

,Imputation_Method,Missing Fraction,NumberOfInstances,NumberOfNumericFeatures,NumberOfCategoricalFeatures,Missing Type,control,Task,Column,result_type,...,Unnamed: 0,name,MajorityClassSize,MinorityClassSize,NumberOfFeatures,NumberOfClasses,Downstream Performance Rank,Data_Constellation,Data_Constellation_full,Performance Difference to Average Best
0,Random Forest,0.01,14980.0,14.0,1.0,MAR,20,1471,V9,downstream_performance_mean,...,28,eeg-eye-state,8257.0,6723.0,15.0,NaN,3.0,MAR - 0.01,MAR - 0.01 - 1471,0.000000
1,Random Forest,0.01,11183.0,6.0,1.0,MAR,20,310,attr2,downstream_performance_mean,...,12,mammography,10923.0,260.0,7.0,NaN,2.0,MAR - 0.01,MAR - 0.01 - 310,0.000000
2,Random Forest,0.01,40768.0,7.0,4.0,MAR,20,881,x3,downstream_performance_mean,...,39,mv,24321.0,16447.0,11.0,NaN,3.0,MAR - 0.01,MAR - 0.01 - 881,0.000000
3,Random Forest,0.01,40768.0,10.0,1.0,MAR,20,901,X2,downstream_performance_mean,...,38,fried,20427.0,20341.0,11.0,NaN,3.0,MAR - 0.01,MAR - 0.01 - 901,0.000000
4,Random Forest,0.10,39948.0,9.0,1.0,MAR,20,1220,keyword_id,downstream_performance_mean,...,36,Click_prediction_small,33220.0,6728.0,10.0,NaN,2.0,MAR - 0.1,MAR - 0.1 - 1220,0.000000
5,Random Forest,0.10,5404.0,5.0,1.0,MAR,20,1489,V1,downstream_performance_mean,...,4,phoneme,3818.0,1586.0,6.0,NaN,2.0,MAR - 0.1,MAR - 0.1 - 1489,0.000000
6,Random Forest,0.10,39366.0,9.0,1.0,MAR,20,251,Cell_Shape_Uniformity,downstream_performance_mean,...,35,BNG(breast-w),25820.0,13546.0,10.0,NaN,4.0,MAR - 0.1,MAR - 0.1 - 251,0.000000
7,Random Forest,0.10,5278.0,7.0,7.0,MAR,20,42192,age,downstream_performance_mean,...,10,compas-two-years,2795.0,2483.0,14.0,NaN,2.0,MAR - 0.1,MAR - 0.1 - 42192,0.000000
8,Random Forest,0.10,4052.0,7.0,1.0,MAR,20,728,Lower_court_disagreement,downstream_performance_mean,...,3,analcatdata_supreme,3081.0,971.0,8.0,NaN,1.0,MAR - 0.1,MAR - 0.1 - 728,0.000000
9,Random Forest,0.30,19020.0,11.0,1.0,MAR,20,1120,fAlpha_,downstream_performance_mean,...,29,MagicTelescope,12332.0,6688.0,12.0,NaN,1.0,MAR - 0.3,MAR - 0.3 - 1120,0.000000


In [159]:
av_improv_model = result_improv['Performance Difference to Average Best'].mean()
print(av_improv_model)

0.00030657098775577196
